In [ ]:
# Install Java
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Download Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

# Extract Spark
!tar -xvzf spark-3.4.1-bin-hadoop3.tgz

# Install Spark NLP and findspark
!pip install -q findspark spark-nlp

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

import findspark
findspark.init()

# ✅ Start Spark NLP session (IMPORTANT!)
import sparknlp
spark = sparknlp.start()
spark

spark-3.4.1-bin-hadoop3/
spark-3.4.1-bin-hadoop3/R/
spark-3.4.1-bin-hadoop3/R/lib/
spark-3.4.1-bin-hadoop3/R/lib/sparkr.zip
spark-3.4.1-bin-hadoop3/R/lib/SparkR/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/R.css
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/00Index.html
spark-3.4.1-bin-hadoop3/R/lib/SparkR/INDEX
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/aliases.rds
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/AnIndex
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/SparkR.rdx
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/SparkR.rdb
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/paths.rds
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/worker.R
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/daemon.R
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/testthat/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/testthat/test_basic.R
spark-3.4.1-bin-hadoop3/R/lib/S

In [ ]:
spark

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving user_input.txt to user_input.txt


In [ ]:

file_path = '/content/user_input.txt'


In [ ]:

df = spark.read.csv('/content/user_input.txt', header=True, inferSchema=True)
df.show(5)


+--------------------------------+
|## **Functional Requirements**  |
+--------------------------------+
|            ### **1. Authenti...|
|            ✅ **Objective:** ...|
|            ✅ **Functionaliti...|
|            - Test weak passw...|
|            - Detect **insecu...|
+--------------------------------+
only showing top 5 rows



In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 46.8 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF

pdf_file_path = '/content/user_input.txt'
pdf_document = fitz.open(pdf_file_path)


pdf_text = ""
for page_num in range(pdf_document.page_count):
    page = pdf_document.load_page(page_num)
    pdf_text += page.get_text()


print(pdf_text[:500])


## **Functional Requirements**  
### **1. Authentication Handling & Session
Management**  
✅ **Objective:** Identify weak authentication
mechanisms, insecure session handling, and credential
exposure.  
✅ **Functionalities:**  
- Test weak passwords via **dictionary attacks**
(`requests`, `bcrypt`).  
- Detect **insecure authentication mechanisms**
(e.g., Basic Auth, JWT flaws).  
- Check for **session fixation & session hijacking**
vulnerabilities.  
- Analyze **cookies for missing security fla


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("PlagiarismDetection") \
    .getOrCreate()


In [ ]:

from pyspark.sql import Row


data = [Row(text=pdf_text)]
df = spark.createDataFrame(data)


df.show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import lit


spark = SparkSession.builder \
    .appName("PlagiarismDetection") \
    .getOrCreate()


data = [(pdf_text,)]
columns = ["text"]


df = spark.createDataFrame(data, columns)



from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, StopWordsCleaner, LemmatizerModel
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")
stopwords_cleaner = StopWordsCleaner().setInputCols(["token"]).setOutputCol("clean_tokens")
lemmatizer = LemmatizerModel.pretrained().setInputCols(["clean_tokens"]).setOutputCol("lemmatized")

# Build the pipeline
pipeline = Pipeline(stages=[document_assembler, tokenizer, stopwords_cleaner, lemmatizer])

# Step 3: Apply the pipeline to the DataFrame
processed_data = pipeline.fit(df).transform(df)

# Step 4: Show the preprocessed data (lemmatized tokens)
processed_data.select("lemmatized.result").show(truncate=False)


lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from pyspark.sql.functions import col

# Extract only the 'result' from the 'lemmatized' column
processed_data = processed_data.withColumn("lemmatized_tokens",
                                           col("lemmatized.result"))

In [ ]:
# Import necessary classes
from pyspark.ml.feature import HashingTF, IDF

# Generate term frequency (TF) features
hashing_tf = HashingTF(inputCol="lemmatized_tokens", outputCol="raw_features")
featurized_data = hashing_tf.transform(processed_data)

# Apply IDF (Inverse Document Frequency)
idf = IDF(inputCol="raw_features", outputCol="features")
idf_model = idf.fit(featurized_data)
tfidf_data = idf_model.transform(featurized_data)

# Show the features (TF-IDF values)
tfidf_data.select("features").show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from pyspark.ml.feature import NGram

# Generate bigrams (n=2)
ngram = NGram(n=2, inputCol="lemmatized_tokens", outputCol="bigrams")
ngram_data = ngram.transform(processed_data)

# Show the bigrams
ngram_data.select("bigrams").show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Save the processed data to a Parquet file and overwrite if it exists
processed_data.write.mode("overwrite").format("parquet").save("/content/processed_data.parquet")


In [ ]:
!pip install beautifulsoup4
!pip install requests

In [ ]:
!pip install pymongo


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 12.6 MB/s eta 0:00:00


In [ ]:
!pip install apache-airflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of apache-airflow-providers-fab to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 6.6 MB/s eta 0:00:00
   ━━

In [ ]:
!pip install kafka-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.4/285.4 kB 2.8 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import string
from nltk.corpus import stopwords
from collections import Counter

# --- STEP 1: Read input document ---
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

# --- STEP 2: Extract keyword-based query ---
def generate_query(text, num_keywords=5):
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    words = text.split()
    stop_words = set(stopwords.words('english'))
    filtered_words = [w for w in words if w not in stop_words]
    most_common = Counter(filtered_words).most_common(num_keywords)
    keywords = [word for word, count in most_common]
    return " ".join(keywords)

# --- STEP 3: Fetch articles from arXiv ---
def fetch_arxiv_articles(query, max_results=5):
    search_url = f'http://export.arxiv.org/api/query?search_query=all:{query}&start=0&max_results={max_results}'
    response = requests.get(search_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'xml')
    articles = []
    for entry in soup.find_all('entry'):
        title = entry.title.text
        authors = [author.find('name').text for author in entry.find_all('author')]
        abstract = entry.summary.text
        url = entry.id.text
        articles.append({
            'title': title,
            'authors': authors,
            'abstract': abstract,
            'url': url
        })
    return articles

# --- MAIN ---
if __name__ == "__main__":
    file_path = "user_input.txt"  # Replace with your file path
    doc_text = read_text_file(file_path)

    generated_query = generate_query(doc_text)
    print(f"🔍 Generated Query: {generated_query}")

    print("\n📄 Fetching Similar Articles from arXiv...\n")
    arxiv_results = fetch_arxiv_articles(generated_query)

    for idx, article in enumerate(arxiv_results, 1):
        print(f"Article {idx}:")
        print(json.dumps(article, indent=2))
        print("-" * 40)


🔍 Generated Query: ✅ security scanning identify oop

📄 Fetching Similar Articles from arXiv...

Article 1:
{
  "title": "Oops!...I think I scanned a malware",
  "authors": [
    "Ben Nassi",
    "Adi Shamir",
    "Yuval Elovici"
  ],
  "abstract": "  This article presents a proof-of-concept illustrating the feasibility of\ncreating a covert channel between a C\\&C server and a malware installed in an\norganization by exploiting an organization's scanner and using it as a means of\ninteraction. We take advantage of the light sensitivity of a flatbed scanner,\nusing a light source to infiltrate data to an organization. We present an\nimplementation of the method for different purposes (even to trigger a\nransomware attack) in various experimental setups using: (1) a laser connected\nto a stand (2) a laser carried by a drone, and (3) a hijacked smart bulb within\nthe targeted organization from a passing car. In our experiments we were able\nto infiltrate data using different types of ligh

In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
with open("/content/user_input.txt", 'r', encoding='utf-8') as f:
    pdf_text = f.read()


In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import string
from nltk.corpus import stopwords
from collections import Counter

# --- STEP 1: Read input document ---
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

# --- STEP 2: Extract keyword-based query ---
def generate_query(text, num_keywords=5):
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    words = text.split()
    stop_words = set(stopwords.words('english'))
    filtered_words = [w for w in words if w not in stop_words]
    most_common = Counter(filtered_words).most_common(num_keywords)
    keywords = [word for word, count in most_common]
    return " ".join(keywords)

# --- STEP 3: Fetch articles from arXiv ---
def fetch_arxiv_articles(query, max_results=5):
    search_url = f'http://export.arxiv.org/api/query?search_query=all:{query}&start=0&max_results={max_results}'
    response = requests.get(search_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'xml')
    articles = []
    for entry in soup.find_all('entry'):
        title = entry.title.text
        authors = [author.find('name').text for author in entry.find_all('author')]
        abstract = entry.summary.text
        url = entry.id.text
        articles.append({
            'title': title,
            'authors': authors,
            'abstract': abstract,
            'url': url
        })
    return articles

# --- MAIN ---
if __name__ == "__main__":
    file_path = "user_input.txt"  # Replace with your file path
    doc_text = read_text_file(file_path)

    generated_query = generate_query(doc_text)
    print(f"🔍 Generated Query: {generated_query}")

    print("\n📄 Fetching Similar Articles from arXiv...\n")
    arxiv_results = fetch_arxiv_articles(generated_query)

    # Save the articles data as a JSON file
    with open('/content/web_dataset.json', 'w') as f:
        json.dump(arxiv_results, f, indent=4)  # Use indent for better formatting

    for idx, article in enumerate(arxiv_results, 1):
        print(f"Article {idx}:")
        print(json.dumps(article, indent=2))
        print("-" * 40)

🔍 Generated Query: ✅ security scanning identify oop

📄 Fetching Similar Articles from arXiv...

Article 1:
{
  "title": "Oops!...I think I scanned a malware",
  "authors": [
    "Ben Nassi",
    "Adi Shamir",
    "Yuval Elovici"
  ],
  "abstract": "  This article presents a proof-of-concept illustrating the feasibility of\ncreating a covert channel between a C\\&C server and a malware installed in an\norganization by exploiting an organization's scanner and using it as a means of\ninteraction. We take advantage of the light sensitivity of a flatbed scanner,\nusing a light source to infiltrate data to an organization. We present an\nimplementation of the method for different purposes (even to trigger a\nransomware attack) in various experimental setups using: (1) a laser connected\nto a stand (2) a laser carried by a drone, and (3) a hijacked smart bulb within\nthe targeted organization from a passing car. In our experiments we were able\nto infiltrate data using different types of ligh

In [ ]:
from pyspark.sql import Row
from pyspark.sql.functions import lit
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import Normalizer
from pyspark.ml.linalg import DenseVector
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
import math

# Sample assumption: user_text already exists from PDF preprocessing
# If not, set user_text = "Your extracted document text"

# Step 2.1: Prepare the web data from arXiv (if already fetched)
# If you have arxiv_results from earlier:
arxiv_df = spark.createDataFrame(
    [Row(text=article["title"] + ". " + article["abstract"], source="arxiv") for article in arxiv_results]
)

# Step 2.2: Convert user text into a DataFrame
user_df = spark.createDataFrame([Row(text=pdf_text, source="user_input")])

# Step 2.3: Combine both into combined_df
combined_df = user_df.union(arxiv_df)

# Step 2.4: Tokenization
tokenizer = Tokenizer(inputCol="text", outputCol="words")
words_data = tokenizer.transform(combined_df)

# Step 2.5: TF and IDF
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10000)
featurized_data = hashingTF.transform(words_data)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idf_model = idf.fit(featurized_data)
rescaled_data = idf_model.transform(featurized_data)

# Step 2.6: Normalize vectors for cosine similarity
normalizer = Normalizer(inputCol="features", outputCol="normFeatures")
normalized_data = normalizer.transform(rescaled_data)

# Step 2.7: Compute Cosine Similarity of user_input with each arxiv result
user_vector = normalized_data.filter(normalized_data.source == "user_input").select("normFeatures").first()["normFeatures"]

# UDF to compute cosine similarity
def cosine_sim(vec):
    return float(vec.dot(user_vector))

cosine_sim_udf = udf(cosine_sim, DoubleType())
scored_df = normalized_data.withColumn("cosine_similarity", cosine_sim_udf("normFeatures"))

# Step 2.8: Show sorted results (excluding self-comparison)
scored_df.filter(scored_df.source != "user_input") \
    .select("text", "cosine_similarity") \
    .orderBy("cosine_similarity", ascending=False) \
    .show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
!pip install nltk

In [ ]:
# STEP 0: INSTALLS & SETUP
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar -xvzf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark spark-nlp nltk beautifulsoup4 requests sentence-transformers

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

import findspark
findspark.init()

import nltk
nltk.download("punkt")
nltk.download("stopwords")

import requests, string, json
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer, util
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col, lit
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
import pandas as pd

spark = SparkSession.builder.appName("PlagiarismDetectionPipeline").getOrCreate()

# STEP 1: USER FILE UPLOAD
from google.colab import files
uploaded = files.upload()
file_path = list(uploaded.keys())[0]

# STEP 2: READ TEXT FILE + EXTRACT KEYWORDS
def read_text_file(fp):
    with open(fp, 'r', encoding='utf-8') as f:
        return f.read()

def generate_query(text, num_keywords=5):
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    words = text.split()
    stop_words = set(stopwords.words('english'))
    filtered = [w for w in words if w not in stop_words]
    keywords = [word for word, count in Counter(filtered).most_common(num_keywords)]
    return " ".join(keywords)

user_text = read_text_file(file_path)
query = generate_query(user_text)

# STEP 3: FETCH SIMILAR ARTICLES FROM ARXIV
def fetch_arxiv_articles(query, max_results=5):
    search_url = f"http://export.arxiv.org/api/query?search_query=all:{query}&start=0&max_results={max_results}"
    response = requests.get(search_url)
    soup = BeautifulSoup(response.content, 'xml')
    articles = []
    for entry in soup.find_all('entry'):
        title = entry.title.text
        authors = [author.find('name').text for author in entry.find_all('author')]
        abstract = entry.summary.text
        url = entry.id.text
        articles.append(f"{title}. {abstract}")
    return articles

arxiv_articles = fetch_arxiv_articles(query)
all_docs = [user_text] + arxiv_articles

# STEP 4: LOAD INTO SPARK
rows = [Row(text=doc) for doc in all_docs]
df = spark.createDataFrame(rows)

# STEP 5: TOKENIZATION + TF-IDF
tokenizer = Tokenizer(inputCol="text", outputCol="words")
words_data = tokenizer.transform(df)
hashing_tf = HashingTF(inputCol="words", outputCol="raw_features")
featurized_data = hashing_tf.transform(words_data)
idf = IDF(inputCol="raw_features", outputCol="features")
idf_model = idf.fit(featurized_data)
tfidf_data = idf_model.transform(featurized_data)

# STEP 6: COSINE SIMILARITY + SENTENCE MATCHING
model = SentenceTransformer('all-MiniLM-L6-v2')

user_sentences = sent_tokenize(user_text)
article_sentences = []
article_ids = []

for i, article in enumerate(arxiv_articles):
    sents = sent_tokenize(article)
    article_sentences.extend(sents)
    article_ids.extend([i + 1] * len(sents))  # i+1 because 0 is the user

user_embeddings = model.encode(user_sentences, convert_to_tensor=True)
article_embeddings = model.encode(article_sentences, convert_to_tensor=True)

cos_scores = util.cos_sim(user_embeddings, article_embeddings)

similar_pairs = []
for i in range(len(user_sentences)):
    for j in range(len(article_sentences)):
        score = cos_scores[i][j].item()
        if score > 0.75:
            similar_pairs.append({
                "user_sentence": user_sentences[i],
                "matched_sentence": article_sentences[j],
                "article_id": article_ids[j],
                "similarity_score": score
            })

# STEP 7: DISPLAY TOP MATCHES
if similar_pairs:
    df_result = pd.DataFrame(similar_pairs).sort_values(by="similarity_score", ascending=False)
    print("\n🧠 Top Similar Sentences Found:")
    display(df_result.head(10))
else:
    print("✅ No significant plagiarism detected.")

# OPTIONAL STEP 8: SAVE TO JSON REPORT
with open("/content/plagiarism_report.json", "w") as f:
    json.dump(similar_pairs, f, indent=2)


spark-3.4.1-bin-hadoop3/
spark-3.4.1-bin-hadoop3/R/
spark-3.4.1-bin-hadoop3/R/lib/
spark-3.4.1-bin-hadoop3/R/lib/sparkr.zip
spark-3.4.1-bin-hadoop3/R/lib/SparkR/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/R.css
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/00Index.html
spark-3.4.1-bin-hadoop3/R/lib/SparkR/INDEX
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/aliases.rds
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/AnIndex
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/SparkR.rdx
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/SparkR.rdb
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/paths.rds
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/worker.R
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/daemon.R
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/testthat/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/testthat/test_basic.R
spark-3.4.1-bin-hadoop3/R/lib/S

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Install dependencies for PDF generation
!apt-get install -y wkhtmltopdf
!pip install pdfkit

# Install Gradio for Web UI
!pip install gradio


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  avahi-daemon geoclue-2.0 glib-networking glib-networking-common
  glib-networking-services gsettings-desktop-schemas iio-sensor-proxy
  libavahi-core7 libavahi-glib1 libdaemon0 libevdev2 libgudev-1.0-0 libhyphen0
  libinput-bin libinput10 libjson-glib-1.0-0 libjson-glib-1.0-common
  libmbim-glib4 libmbim-proxy libmd4c0 libmm-glib0 libmtdev1 libnl-genl-3-200
  libnotify4 libnss-mdns libproxy1v5 libqmi-glib5 libqmi-proxy libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5positioning5 libqt5printsupport5
  libqt5qml5 libqt5qmlmodels5 libqt5quick5 libqt5sensors5 libqt5svg5
  libqt5webchannel5 libqt5webkit5 libqt5widgets5 libsoup2.4-1
  libsoup2.4-common libudev1 libwacom-bin libwacom-common libwacom9 libwoff1
  libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-render-util0 libxcb-util1
  libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1 

In [ ]:
from google.colab import files

# Upload the document
uploaded = files.upload()

# Read the document content
file_path = list(uploaded.keys())[0]  # Get the filename
with open(file_path, 'r') as file:
    user_document = file.read()

print("Document successfully uploaded and read!")


Saving user_input.txt to user_input (1).txt
Document successfully uploaded and read!


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Download the missing Punkt model

# Tokenize and remove stopwords
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text.lower())  # Tokenize and convert to lowercase
    filtered_tokens = [word for word in word_tokens if word.isalnum() and word not in stop_words]
    return " ".join(filtered_tokens)

processed_document = preprocess_text(user_document)
print("Preprocessing completed!")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
import requests
from bs4 import BeautifulSoup

# Fetch articles from ArXiv based on a query
def fetch_arxiv_articles(query, max_results=5):
    search_url = f'http://export.arxiv.org/api/query?search_query=all:{query}&start=0&max_results={max_results}'
    response = requests.get(search_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'xml')
    articles = []
    for entry in soup.find_all('entry'):
        title = entry.title.text
        authors = [author.find('name').text for author in entry.find_all('author')]
        abstract = entry.summary.text
        url = entry.id.text
        articles.append({'title': title, 'authors': authors, 'abstract': abstract, 'url': url})
    return articles

# Generate query from the document text
query = ' '.join(processed_document.split()[:5])  # First 5 words of the document as the query
articles = fetch_arxiv_articles(query)
print(f"Found {len(articles)} articles!")


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize sentence transformer model (BERT-based)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode the user's document and fetched articles
doc_embedding = model.encode([processed_document])
article_embeddings = model.encode([article['abstract'] for article in articles])

# Compute cosine similarity between document and articles
similarity_scores = cosine_similarity(doc_embedding, article_embeddings)

# Show the similarity scores
for idx, score in enumerate(similarity_scores[0]):
    print(f"Article {idx+1}: Similarity Score = {score:.4f}")


In [ ]:
!pip install fpdf

In [ ]:
!pip install reportlab

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import datetime

# Generate report using reportlab
def generate_report_with_reportlab(similarity_scores, articles, user_document):
    pdf_output_path = "/content/plagiarism_report_reportlab.pdf"

    c = canvas.Canvas(pdf_output_path, pagesize=letter)

    # Title
    c.setFont("Helvetica-Bold", 16)
    c.drawString(200, 750, "Plagiarism Detection Report")

    # Date
    c.setFont("Helvetica", 12)
    c.drawString(200, 730, f"Date: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # User document preview
    c.drawString(50, 700, "User Document Preview:")
    c.setFont("Helvetica", 10)
    c.drawString(50, 680, f"{user_document[:500]}...")  # Preview first 500 characters

    # Articles and similarity results
    y_position = 650
    c.setFont("Helvetica-Bold", 12)
    for idx, score in enumerate(similarity_scores[0]):
        c.drawString(50, y_position, f"Article {idx+1}: Similarity Score = {score:.4f}")
        y_position -= 20
        c.setFont("Helvetica", 10)
        c.drawString(50, y_position, f"Title: {articles[idx]['title']}")
        y_position -= 20
        c.drawString(50, y_position, f"Abstract: {articles[idx]['abstract']}")
        y_position -= 20
        c.drawString(50, y_position, f"URL: {articles[idx]['url']}")
        y_position -= 40

    c.save()
    return pdf_output_path

# Generate the report with reportlab
pdf_report_path_rl = generate_report_with_reportlab(similarity_scores, articles, user_document)
print(f"Plagiarism report saved to: {pdf_report_path_rl}")


In [ ]:
pdf_report_path_rl = generate_report_with_reportlab(similarity_scores, articles, user_document)
print(f"Plagiarism report saved to: {pdf_report_path_rl}")


In [ ]:
from google.colab import files
files.download(pdf_report_path_rl)  # For ReportLab


In [ ]:
!pip install Flask Flask-Uploads


In [ ]:
!pip install flask-ngrok
